In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import warnings
from textblob import TextBlob
from wordcloud import WordCloud
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
movies=pd.read_csv('./data/movies.csv', index_col=0)
movies.head()

,title,overview,original_language,vote_count,vote_average
0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9
1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2
2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1
3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1
4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9


In [3]:
movies.isnull().sum()

title                 0
overview             30
original_language     0
vote_count            0
vote_average          0
dtype: int64

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              10000 non-null  object 
 1   overview           9970 non-null   object 
 2   original_language  10000 non-null  object 
 3   vote_count         10000 non-null  int64  
 4   vote_average       10000 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 468.8+ KB


In [4]:
movies.dropna(inplace=True)

In [6]:
movies.isnull().sum()

title                0
overview             0
original_language    0
vote_count           0
vote_average         0
dtype: int64

In [17]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9970 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              9970 non-null   object 
 1   overview           9970 non-null   object 
 2   original_language  9970 non-null   object 
 3   vote_count         9970 non-null   int64  
 4   vote_average       9970 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 467.3+ KB


In [30]:
# Content-Based Filtering (using movie overviews)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['overview'].fillna(''))
cosine_sim_content = cosine_similarity(tfidf_matrix)

In [18]:

# Build the item-item matrix using the DataFrame index
item_item_matrix = movies.pivot(columns='title', values='vote_average')

# Calculate the cosine similarity between the items (movies) based on the item-item matrix
cosine_sim_collaborative = cosine_similarity(item_item_matrix.fillna(0))

In [12]:
movies.head()

,title,overview,original_language,vote_count,vote_average,movies_id,movie_id
0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9,None,0
1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2,None,1
2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1,None,2
3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1,None,3
4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9,None,4


In [31]:
# Content-Enhanced Recommender Function
def content_enhanced_recommender(movie_title, movie_data, cosine_sim_content, cosine_sim_collaborative):
    target_movie_index = movies.index[movies['title'] == movie_title].tolist()[0]
    
    # Get content-based similarity scores for the target movie
    content_scores = cosine_sim_content[target_movie_index]

    # Get collaborative similarity scores for movies similar to the target movie based on ratings
    collaborative_scores = cosine_sim_collaborative[target_movie_index]

    # Combine content-based and collaborative scores
    hybrid_scores = content_scores + collaborative_scores

    # Get indices of top 5 movies based on hybrid scores
    top_indices = hybrid_scores.argsort()[::-1][:5]

    # Get the titles of the top 5 recommended movies
    recommended_movies = movies.iloc[top_indices]['title']
    return recommended_movies    


In [5]:
# Build the item-item matrix using the DataFrame index
item_item_matrix = movies.pivot(columns='title', values='vote_average')


In [33]:
# Example target movie (you should replace this with the actual movie title)
target_movie_title = 'Bad Boys for Life'
recommended_movies = content_enhanced_recommender(target_movie_title, movies, cosine_sim_content, cosine_sim_collaborative)
print(recommended_movies)

2            Bad Boys for Life
437                Bad Boys II
8946           Miami Supercops
735                   Scarface
8619    Running with the Devil
Name: title, dtype: object
